In [2]:
!pip install wbdata duckdb

import wbdata
import duckdb
import pandas as pd
import datetime
countries = ['USA', 'GB', 'IN', 'KE', 'BR']

indicators = {
    'SP.DYN.LE00.IN': 'life_expectancy',
    'SH.DYN.MORT': 'under5_mortality',
    'SP.ADO.TFRT': 'adolescent_fertility',
    'SE.SEC.ENRR': 'secondary_school_enrollment'
}

df = wbdata.get_dataframe(indicators, country=countries)
df = df.reset_index()

df['year'] = pd.to_datetime(df['date']).dt.year

df = df[(df['year'] >= 2000) & (df['year'] <= 2023)]

  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 14.7 MB/s eta 0:00:0000:0100:01
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
  Attempting uninstall: pytz
    Found existing installation: pytz 2024.1
    Uninstalling pytz-2024.1:
      Successfully uninstalled pytz-2024.1
  Attempting uninstall: tabulate
    Found existing installation: tabulate 0.9.0
    Uninstalling tabulate-0.9.0:
      Successfully uninstalled tabulate-0.9.0


In [ ]:
duckdb.sql("CREATE OR REPLACE VIEW wdi_raw AS SELECT * FROM df")

df_clean = duckdb.sql("""
SELECT
  country,
  year,
  MAX(life_expectancy) AS life_expectancy,
  MAX(under5_mortality) AS under5_mortality,
  MAX(adolescent_fertility) AS adolescent_fertility,
  MAX(secondary_school_enrollment) AS secondary_school_enrollment
FROM wdi_raw
GROUP BY country, year
ORDER BY country, year
""").df()

df_clean.head()

,country,year,life_expectancy,under5_mortality,adolescent_fertility,secondary_school_enrollment
0,Brazil,2000,69.584000,34.5,85.611,NaN
1,Brazil,2001,69.980000,32.2,84.155,NaN
2,Brazil,2002,70.396000,30.1,81.131,NaN
3,Brazil,2003,70.884000,28.1,78.025,NaN
4,Brazil,2004,71.361000,26.3,76.900,NaN
...,...,...,...,...,...,...
115,United States,2019,78.787805,6.5,16.202,100.063431
116,United States,2020,76.980488,6.5,14.719,100.509819
117,United States,2021,76.329268,6.5,13.724,101.192490
118,United States,2022,77.434146,6.5,13.628,97.473488


In [8]:
df_clean.to_csv("population_dynamics_clean.csv", index=False)